In [4]:
import json
from web3 import Web3
from solcx import compile_standard, install_solc
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()

False

In [34]:
with open(
    "/home/sahilarora/capstone-19BCE1366-2023/contract/SimpleStorageContract.sol",
    "r",
) as file:
    simple_storage_file = file.read()

In [35]:
install_solc("0.6.0")

Version('0.6.0')

In [36]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorageContract.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.6.0",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)



In [37]:
bytecode = compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"][
    "evm"
]["bytecode"]["object"]

In [38]:
abi = json.loads(
    compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"]["metadata"]
)["output"]["abi"]

In [45]:
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [46]:
chain_id = 1337
my_address = "0x6A426eBA773251ab38293dF600966B4d0680B9b5"
# private_key = os.getenv("PRIVATE_KEY")
private_key = "0xb28074e5a10e4e7353aafa1deb4593df4c0dcf49848c3c4b275c0b37ee8aa184"

In [60]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

In [61]:
nonce = w3.eth.getTransactionCount(my_address)

In [62]:
tx = SimpleStorage.constructor().buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": nonce,
    }
)

In [63]:
signed_tx = w3.eth.account.signTransaction(tx, private_key=private_key)

In [64]:
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [65]:
print(f"Contract deployed to {tx_receipt.contractAddress}")

Contract deployed to 0x695FcFCf4F3877A1272a81ed160acDFAFE756962


In [66]:
simple_storage = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

In [67]:
print(f"Initial stored value at Retrieve {simple_storage.functions.retrieve().call()}")

Initial stored value at Retrieve 0


In [68]:
new_transaction = simple_storage.functions.store(25).buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": nonce + 1,
    }
)

In [69]:
signed_new_txn = w3.eth.account.sign_transaction(
    new_transaction, private_key=private_key
)

In [73]:
tx_new_hash = w3.eth.send_raw_transaction(signed_new_txn.rawTransaction)
print("Sending new transaction...")
tx_new_receipt = w3.eth.wait_for_transaction_receipt(tx_new_hash)

ValueError: {'message': "the tx doesn't have the correct nonce. account has nonce of: 4 tx has nonce of: 3", 'stack': "Error: the tx doesn't have the correct nonce. account has nonce of: 4 tx has nonce of: 3\n    at TransactionPool.prepareTransaction (/usr/lib/node_modules/ganache/dist/node/1.js:2:166240)", 'code': -32000}

In [72]:
print(f"New stored value at Retrieve {simple_storage.functions.retrieve().call()}")

New stored value at Retrieve 25
